In [ ]:
import tensorflow as tf; print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices("GPU")))

In [1]:
import sys
sys.path.append('/usr/local/nvidia/bin:/usr/local/cuda/bin')
sys.path.append('./.local/lib/python3.8/site-packages')
sys.path.append('/usr/lib/python3/dist-packages')

from keras import losses, metrics
from keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, Conv1D, AveragePooling1D, MaxPooling1D, GlobalMaxPooling1D, BatchNormalization, Dropout, Activation, Input
from keras.layers import GlobalAveragePooling1D, Flatten, SeparableConv1D, concatenate, Add
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import auc, classification_report, confusion_matrix, accuracy_score, roc_curve, roc_auc_score, f1_score, precision_recall_curve
import tensorflow as tf
import itertools as it
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats, interp
import os, sys, pickle, shutil
import pandas as pd
import numpy as np
import random, datetime, time
from sklearn.model_selection import KFold
#from keras.utils.training_utils import multi_gpu_model


# tensorflow 사용 시 seed 고정
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)
    
SEED = 98
GPU = 0
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]= f"{GPU}"  

print(datetime.datetime.now())

2023-09-22 14:20:36.258352: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-22 14:20:37.094208: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/nvidia/cuda_nvrtc/lib:/opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/nvidia/cudnn/lib:/opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/nvidia/cublas/lib:/opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/nvidia/cudnn/lib:/opt/anaconda-3-2020.02/envs/hskim/lib/:/usr/local/cudnn-112-81133/lib64:/u

2023-09-22 14:20:38.468700


In [2]:
tf.config.list_logical_devices('GPU')

2023-09-22 00:17:45.796038: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-22 00:17:47.184134: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21957 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:81:00.0, compute capability: 8.6
2023-09-22 00:17:47.184801: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 21956 MB memory:  -> device: 1, name: NVIDIA RTX A5000, pci bus id: 0000:82:00.0, compute capability: 8.6


[LogicalDevice(name='/device:GPU:0', device_type='GPU'),
 LogicalDevice(name='/device:GPU:1', device_type='GPU')]

# Input loading

## signals only

In [3]:
SRATE = 500
ECG_FILT = 'bandpass'
TRAIN = 'train'
ADULT = 'adult'
NORM = 'z-norm'
GENDER = 'male'


hyper_path = f'SRATE{SRATE}_ECG-{ECG_FILT}_{NORM}_{ADULT}_{TRAIN}'
input_path = f'dataset/{hyper_path}/'

with np.load(input_path+'x.npz', allow_pickle=True) as f:
    x_train = f['x_train']
    x_test = f['x_test']
        
with np.load(input_path+'y.npz', allow_pickle=True) as f:
    y_train = f['y_train']
    y_test = f['y_test']
    
with np.load(input_path+'c.npz', allow_pickle=True) as f:
    c_train = f['c_train']
    c_test = f['c_test']  
    
with np.load(input_path+'g.npz', allow_pickle=True) as f:
    g_train = f['g_train']
    g_test = f['g_test']        
    
print(f'x train shape: {x_train.shape}')
print(f'x test shape: {x_test.shape}')

x train shape: (27844, 5000, 12)
x test shape: (6965, 5000, 12)


In [4]:
hyper_path = f'SRATE{SRATE}_ECG-{ECG_FILT}_{NORM}_{ADULT}_{GENDER}_{TRAIN}'

train_mask = (g_train == 1) if GENDER == 'male' else (g_train == 0)
test_mask = (g_test == 1) if GENDER == 'male' else (g_test == 0)

x_train = x_train[train_mask]
y_train = y_train[train_mask]
c_train = c_train[train_mask]
g_train = g_train[train_mask]

x_test = x_test[test_mask]
y_test = y_test[test_mask]
c_test = c_test[test_mask]
g_test = g_test[test_mask]

print(f'x train shape: {x_train.shape}')
print(f'x test shape: {x_test.shape}')

x train shape: (13378, 5000, 12)
x test shape: (3390, 5000, 12)


In [5]:
y_train = y_train / 100
y_test = y_test / 100

## signal+features

In [2]:
SRATE = 500
ECG_FILT = 'bandpass'
TRAIN = 'train'
ADULT = 'adult'
NORM = 'z-norm'
GENDER = 'male'

hyper_path = f'SRATE{SRATE}_ECG-{ECG_FILT}_{NORM}_{ADULT}_{TRAIN}'
input_path = f'dataset/{hyper_path}/signal+features/'

with np.load(input_path+'x.npz', allow_pickle=True) as f:
    x_train = f['x_train']
    x_test = f['x_test']
     
with np.load(input_path+'feats.npz', allow_pickle=True) as f:
    feat_train = f['feat_train']
    feat_test = f['feat_test']
        
with np.load(input_path+'y.npz', allow_pickle=True) as f:
    y_train = f['y_train']
    y_test = f['y_test']
    
with np.load(input_path+'c.npz', allow_pickle=True) as f:
    c_train = f['c_train']
    c_test = f['c_test']  
    
with np.load(input_path+'g.npz', allow_pickle=True) as f:
    g_train = f['g_train']
    g_test = f['g_test']        
    
print(f'x train shape: {x_train.shape}')
print(f'feat train shape: {feat_train.shape}')
print(f'x test shape: {x_test.shape}')
print(f'feat test shape: {feat_test.shape}')

hyper_path = f'SRATE{SRATE}_ECG-{ECG_FILT}_{NORM}_{ADULT}_{GENDER}_{TRAIN}'

train_mask = (g_train == 1) if GENDER == 'male' else (g_train == 0)
test_mask = (g_test == 1) if GENDER == 'male' else (g_test == 0)

x_train = x_train[train_mask]
feat_train = feat_train[train_mask]
y_train = y_train[train_mask]
c_train = c_train[train_mask]
g_train = g_train[train_mask]

x_test = x_test[test_mask]
feat_test = feat_test[test_mask]
y_test = y_test[test_mask]
c_test = c_test[test_mask]
g_test = g_test[test_mask]

print(f'x train shape: {x_train.shape}')
print(f'feat train shape: {feat_train.shape}')
print(f'x test shape: {x_test.shape}')
print(f'feat test shape: {feat_test.shape}')


SCALE_Y = 100 if ADULT == 'adult' else 20
y_train = y_train / SCALE_Y
y_test = y_test / SCALE_Y

print(f'train: {np.mean(y_train)}, test: {np.mean(y_test)}')

x train shape: (27824, 5000, 12)
feat train shape: (27824, 55)
x test shape: (6962, 5000, 12)
feat test shape: (6962, 55)
x train shape: (13358, 5000, 12)
feat train shape: (13358, 55)
x test shape: (3387, 5000, 12)
feat test shape: (3387, 55)
train: 0.6191248297691345, test: 0.6184705495834351


# Settings

In [5]:
# folder
nfold = 4  # 각각의 hyperparameter에 대해 k-fold 를 시행하고 평균을 구한다.
ntest = 500
rootdir = f"randomSearch/{hyper_path}/CNN_4layers_large-kernels_age%100(sigmoid)_{nfold}fold(2split)_test{ntest}"

if not os.path.exists(f"randomSearch/{hyper_path}"):
    os.mkdir(f"randomSearch/{hyper_path}")

if not os.path.exists(rootdir):
    os.mkdir(rootdir)

# 모델에 대한 정보 txt로 저장
f = open(f'{rootdir}/README.txt', 'w')
f.write(f'model: 1D CNN 4 layers, regression')
f.write(f'input: ECG of 10 second, output: age')
f.close()
    

# test_settings
layer_settings, test_settings = [], []


# hyperparamters
globalpool_opts = ['max','ave']

# hyperparamters pool
filt_opts = [16, 32, 64, 128] # num of filters(kernel)
stride_opts = [1,2,4]  # other opts: stride = (kernel-1)/2
kernel_opts = range(7,20,2) # kernel size
pool_size = 2
dropout_opts  = [0, 0.1, 0.2, 0.3, 0.4, 0.5] # dropout rate
dense_opts = [0, 16, 32, 64, 128]
BATCH_SIZE = [32, 64, 128, 256, 512]
lr_opts = [0.001, 0.002, 0.0005]

print('start making test settings...', end='', flush=True)
# test settings
nfilt, kernels, strides = [], [], []
for i in range(5):
    nfilt.append(0)
    kernels.append(0)
    strides.append(0)

for nfilter in filt_opts:
    for kernel in kernel_opts:
        for stride in stride_opts:
        #layer_settings.append([nfilter, kernel, int((kernel-1)/2)])       
            layer_settings.append([nfilter, kernel, stride])
    
for dense_node in dense_opts:
    for dropout_cnn in dropout_opts:
        for dropout_fc in dropout_opts:
            for batch_size in BATCH_SIZE:
                for learning_rate in lr_opts:
                    test_settings.append([dense_node, dropout_cnn, dropout_fc, batch_size, learning_rate])                                   

                        
print('done')
print(datetime.datetime.now())

start making test settings...done
2023-09-21 02:50:09.717600


## Random Searching

## 1d-cnn

In [ ]:
from keras import metrics
from tensorflow.keras.callbacks import LearningRateScheduler

seed_everything(SEED)

# random search for hyperparameter
ntrial = ntest
train_errs, val_errs = [] ,[]
test_acc, test_roc, test_prc = [], [], []
#test_rmse, test_mae, test_auc = [], [], []
random_settings = []


for itrial in range(ntrial):
    # grid search
    # test_setting = test_settings[itrial]

    # random search
    print('random search {}/{}'.format(itrial, ntrial))
    
    # total conv layers of the model
    n_conv = random.choice([2,3,4]) 
    # test settings
    for i in range(n_conv):
        nfilt[i], kernels[i], strides[i] = random.choice(layer_settings)
    dense_node, dropout_cnn, dropout_fc, batch_size, learning_rate = random.choice(test_settings)
    
    if itrial < 0:
        continue
    

    # 이번 옵션에 대한 결과 디렉토리
    odir_f = f'batch{batch_size},'
    for i in range(n_conv):
        odir_f += f'layer{i+1}:c{nfilt[i]}filt{kernels[i]}str{strides[i]},'
    odir_f += f'1conv,dropout{dropout_cnn},dnodes{dense_node},dropout{dropout_fc}'#,lr{learning_rate}'
    random_settings.append(odir_f)
    
    odir = rootdir + '/' + odir_f
    if not os.path.exists(odir):
        os.mkdir(odir)

        
    # model validation (VGG)
    out_shape = x_train.shape[1]
    for i in range(n_conv):
        out_shape = out_shape / pool_size / strides[i]
    if out_shape < 1:
        print('non-valid structure')
        os.rmdir(odir)
        itrial -= 1
        continue
        
    #mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/GPU:0","/GPU:1","/GPU:2","/GPU:3"])
    #with mirrored_strategy.scope(): 
    
    with tf.device(f'/gpu:{GPU}'):
        # build a model
        inp = Input(shape=(x_train.shape[1],x_train.shape[2]))
        out = inp


        # VGC block
        for i in range(n_conv):
            out = Conv1D(filters=nfilt[i], kernel_size=kernels[i], strides=strides[i], padding='same', activation=None)(out)
            out = BatchNormalization()(out)
            out = Activation('relu')(out)
            out = MaxPooling1D(pool_size)(out)


        # globalpooling vs flattening vs 1x1 convolution
        #elif globalpool_opt == 'ave':
        #    out = GlobalAveragePooling1D()(out)
        out = Conv1D(filters=1, kernel_size=1)(out)
        out = Flatten() (out)



        if dense_node != 0:
            out = Dropout(dropout_cnn)(out)
            out = Dense(dense_node, activation='relu')(out)
        out = Dropout(dropout_fc)(out)
        out = Dense(1, activation='sigmoid')(out)


        model = Model(inputs=[inp], outputs=[out])
        model.save_weights(f'{odir}/initial_weights.hdf5')


        # 4-fold cv
        kfold = KFold(nfold)
        tprs, aucs, prs = [], [], []
        test_rmse, test_mae = [], []
        f1_scores, thvals = [], []
        mean_fpr = np.linspace(0,1,100)
        mean_recall = np.linspace(0,1,100)

        switch = 0
        caseids_train = np.unique(c_train)
        for fold, (c_cv_trains_mask, c_cv_test_mask) in enumerate(kfold.split(caseids_train)):
            c_cv_trains = caseids_train[c_cv_trains_mask]
            
            cv_train_mask = np.isin(c_train, c_cv_trains)
            cv_val_mask = ~cv_train_mask
            
            X_train = x_train[cv_train_mask]
            X_val = x_train[cv_val_mask]

            Y_train = y_train[cv_train_mask]
            Y_val = y_train[cv_val_mask]


            # model 학습
            try:
                # learning scheduler
                def step_decay(epoch):
                    start = 1e-3
                    drop = 0.1
                    epochs_drop = 10
                    lr = start * (drop ** np.floor((epoch)/epochs_drop))
                    return lr
                lr_scheduler = LearningRateScheduler(step_decay, verbose=1)
                
                weightcache = f"{odir}/weights_{fold}.hdf5"
                #model = multi_gpu_model(model, gpus=4)
                model.compile(loss='mean_squared_error', optimizer=Adam(lr=lr_scheduler, weight_decay=1e-4), metrics=['mean_absolute_error'])
                hist = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                        callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                    EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')])

                model.load_weights(weightcache)
                y_pred = model.predict(x_test).flatten()

                # MAE 계산
                model_err = metrics.MeanAbsoluteError()
                model_err.update_state(y_test, y_pred)
                mae_val = model_err.result().numpy()
                test_mae.append(mae_val)

                # RMSE 계산
                model_err = metrics.RootMeanSquaredError() 
                model_err.update_state(y_test, y_pred)
                rmse_val = model_err.result().numpy()
                test_rmse.append(rmse_val)


                print(f' ###{fold} fold : val mae {mae_val:.2f}, val rmse {rmse_val:.2f}###')
                tf.keras.backend.clear_session()
                model.load_weights(f'{odir}/initial_weights.hdf5')

            except Exception as e:
                print(e)
                switch = 1
                shutil.rmtree(odir)
                itrial -= 1
                break
        ###
        if switch:
            switch = 0
            tf.keras.backend.clear_session()
            continue

    # 

    # RMSE 계산
    mean_rmse = np.mean(test_rmse)
    std_rmse = np.std(test_rmse)
    
    mean_mae = np.mean(test_mae)
    std_mae = np.std(test_mae)

    max_idx = test_mae.index(min(test_mae))
    

    print(f'mae{mean_mae*100:.2f}+-{std_mae*100:.2f}_rmse{mean_rmse*100:.2f}+-{std_rmse*100:.2f}')
    open(odir+"/model.json", "wt").write(model.to_json())

    os.rename(odir, rootdir+f'/mae{mean_mae*100:.2f}+-{std_mae*100:.2f}_rmse{mean_rmse*100:.2f}+-{std_rmse*100:.2f}_max{max_idx}__{odir_f}')
    tf.keras.backend.clear_session()


random search 0/500


2023-09-21 02:50:29.202971: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-21 02:50:29.694842: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22325 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:81:00.0, compute capability: 8.6


Epoch 1/100


2023-09-21 02:50:37.031032: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8904
2023-09-21 02:50:37.498391: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-09-21 02:50:37.590977: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-09-21 02:50:37.628296: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x1df56910 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-21 02:50:37.628327: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA RTX A5000, Compute Capability 8.6
2023-09-21 02:50:37.635399: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-21 02:50:37.707936: I tensor

314/314 [==============================] - ETA: 0s - loss: 0.0195 - mean_absolute_error: 0.1087
Epoch 1: val_loss improved from inf to 0.02124, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers_large-kernels_age%100(sigmoid)_4fold(2split)_test500/batch32,layer1:c128filt13str2,layer2:c16filt9str4,layer3:c64filt9str1,1conv,dropout0.1,dnodes64,dropout0/weights_0.hdf5
314/314 [==============================] - 12s 19ms/step - loss: 0.0195 - mean_absolute_error: 0.1087 - val_loss: 0.0212 - val_mean_absolute_error: 0.1155
Epoch 2/100
312/314 [============================>.] - ETA: 0s - loss: 0.0146 - mean_absolute_error: 0.0944
Epoch 2: val_loss did not improve from 0.02124
314/314 [==============================] - 5s 15ms/step - loss: 0.0146 - mean_absolute_error: 0.0944 - val_loss: 0.0221 - val_mean_absolute_error: 0.1141
Epoch 3/100
313/314 [============================>.] - ETA: 0s - loss: 0.0134 - mean_absolute_error: 0.0906
Epoch 3: val_loss did no

...

Epoch 1/100
314/314 [==============================] - ETA: 0s - loss: 0.0270 - mean_absolute_error: 0.1300
Epoch 1: val_loss improved from inf to 0.01671, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers_large-kernels_age%100(sigmoid)_4fold(2split)_test500/batch32,layer1:c16filt15str1,layer2:c32filt17str2,layer3:c16filt11str2,1conv,dropout0,dnodes16,dropout0.5/weights_0.hdf5
314/314 [==============================] - 9s 17ms/step - loss: 0.0270 - mean_absolute_error: 0.1300 - val_loss: 0.0167 - val_mean_absolute_error: 0.1033
Epoch 2/100
310/314 [============================>.] - ETA: 0s - loss: 0.0183 - mean_absolute_error: 0.1076
Epoch 2: val_loss did not improve from 0.01671
314/314 [==============================] - 4s 14ms/step - loss: 0.0183 - mean_absolute_error: 0.1074 - val_loss: 0.0177 - val_mean_absolute_error: 0.1016
Epoch 3/100
311/314 [============================>.] - ETA: 0s - loss: 0.0161 - mean_absolute_error: 0.0998
Epoch 3: val

Epoch 1/100
311/314 [============================>.] - ETA: 0s - loss: 0.0292 - mean_absolute_error: 0.1350
Epoch 1: val_loss improved from inf to 0.01710, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers_large-kernels_age%100(sigmoid)_4fold(2split)_test500/batch32,layer1:c16filt15str1,layer2:c32filt17str2,layer3:c16filt11str2,1conv,dropout0,dnodes16,dropout0.5/weights_1.hdf5
314/314 [==============================] - 9s 16ms/step - loss: 0.0292 - mean_absolute_error: 0.1350 - val_loss: 0.0171 - val_mean_absolute_error: 0.1042
Epoch 2/100
314/314 [==============================] - ETA: 0s - loss: 0.0191 - mean_absolute_error: 0.1096
Epoch 2: val_loss did not improve from 0.01710
314/314 [==============================] - 4s 13ms/step - loss: 0.0191 - mean_absolute_error: 0.1096 - val_loss: 0.0235 - val_mean_absolute_error: 0.1180
Epoch 3/100
314/314 [==============================] - ETA: 0s - loss: 0.0166 - mean_absolute_error: 0.1017
Epoch 3: val

Epoch 1/100
314/314 [==============================] - ETA: 0s - loss: 0.0268 - mean_absolute_error: 0.1298
Epoch 1: val_loss improved from inf to 0.02071, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers_large-kernels_age%100(sigmoid)_4fold(2split)_test500/batch32,layer1:c16filt15str1,layer2:c32filt17str2,layer3:c16filt11str2,1conv,dropout0,dnodes16,dropout0.5/weights_2.hdf5
314/314 [==============================] - 8s 17ms/step - loss: 0.0268 - mean_absolute_error: 0.1298 - val_loss: 0.0207 - val_mean_absolute_error: 0.1170
Epoch 2/100
312/314 [============================>.] - ETA: 0s - loss: 0.0188 - mean_absolute_error: 0.1088
Epoch 2: val_loss improved from 0.02071 to 0.01513, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers_large-kernels_age%100(sigmoid)_4fold(2split)_test500/batch32,layer1:c16filt15str1,layer2:c32filt17str2,layer3:c16filt11str2,1conv,dropout0,dnodes16,dropout0.5/weights_2.hdf5
314/314 

Epoch 1/100
312/314 [============================>.] - ETA: 0s - loss: 0.0265 - mean_absolute_error: 0.1294
Epoch 1: val_loss improved from inf to 0.01936, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers_large-kernels_age%100(sigmoid)_4fold(2split)_test500/batch32,layer1:c16filt15str1,layer2:c32filt17str2,layer3:c16filt11str2,1conv,dropout0,dnodes16,dropout0.5/weights_3.hdf5
314/314 [==============================] - 8s 16ms/step - loss: 0.0265 - mean_absolute_error: 0.1294 - val_loss: 0.0194 - val_mean_absolute_error: 0.1079
Epoch 2/100
311/314 [============================>.] - ETA: 0s - loss: 0.0187 - mean_absolute_error: 0.1085
Epoch 2: val_loss improved from 0.01936 to 0.01585, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers_large-kernels_age%100(sigmoid)_4fold(2split)_test500/batch32,layer1:c16filt15str1,layer2:c32filt17str2,layer3:c16filt11str2,1conv,dropout0,dnodes16,dropout0.5/weights_3.hdf5
314/314 

Epoch 1/100
157/157 [==============================] - ETA: 0s - loss: 0.0379 - mean_absolute_error: 0.1505
Epoch 1: val_loss improved from inf to 0.02206, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers_large-kernels_age%100(sigmoid)_4fold(2split)_test500/batch64,layer1:c64filt7str1,layer2:c64filt19str4,layer3:c16filt9str2,1conv,dropout0.2,dnodes16,dropout0.4/weights_0.hdf5
157/157 [==============================] - 7s 29ms/step - loss: 0.0379 - mean_absolute_error: 0.1505 - val_loss: 0.0221 - val_mean_absolute_error: 0.1221
Epoch 2/100
156/157 [============================>.] - ETA: 0s - loss: 0.0201 - mean_absolute_error: 0.1129
Epoch 2: val_loss improved from 0.02206 to 0.01504, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers_large-kernels_age%100(sigmoid)_4fold(2split)_test500/batch64,layer1:c64filt7str1,layer2:c64filt19str4,layer3:c16filt9str2,1conv,dropout0.2,dnodes16,dropout0.4/weights_0.hdf5
157/157 

In [2]:
tf.keras.backend.clear_session()

## 1d-cnn + feat concat

In [3]:
# folder
nfold = 4  # 각각의 hyperparameter에 대해 k-fold 를 시행하고 평균을 구한다.
ntest = 500
rootdir = f"randomSearch/{hyper_path}/CNN_4layers+feats_kernel(7~19)_age%{SCALE_Y}(sigmoid)_mse(decay)_{nfold}fold_test{ntest}"

if not os.path.exists(f"randomSearch/{hyper_path}"):
    os.mkdir(f"randomSearch/{hyper_path}")

if not os.path.exists(rootdir):
    os.mkdir(rootdir)

# 모델에 대한 정보 txt로 저장
f = open(f'{rootdir}/README.txt', 'w')
f.write(f'model: 1D CNN 4 layers, regression')
f.write(f'input: ECG of 10 second, output: age')
f.close()
    

# test_settings
layer_settings, test_settings = [], []


# hyperparamters
globalpool_opts = ['max','ave']

# hyperparamters pool
filt_opts = [16, 32, 64] # num of filters(kernel)
stride_opts = [2,4]  # other opts: stride = (kernel-1)/2
kernel_opts = range(7,20,2) # kernel size
pool_size = 2
dropout_opts  = [0, 0.1, 0.2, 0.3, 0.4, 0.5] # dropout rate
dense_opts = [0, 16, 32, 64, 128]
BATCH_SIZE = [32, 64, 128, 256]
lr_opts = [0.001, 0.002, 0.0005]

print('start making test settings...', end='', flush=True)
# test settings
nfilt, kernels, strides = [], [], []
for i in range(5):
    nfilt.append(0)
    kernels.append(0)
    strides.append(0)

for nfilter in filt_opts:
    for kernel in kernel_opts:
        for stride in stride_opts:
        #layer_settings.append([nfilter, kernel, int((kernel-1)/2)])       
            layer_settings.append([nfilter, kernel, stride])
    
for dense_node in dense_opts:
    for dropout_cnn in dropout_opts:
        for dropout_fc in dropout_opts:
            for batch_size in BATCH_SIZE:
                for learning_rate in lr_opts:
                    test_settings.append([dense_node, dropout_cnn, dropout_fc, batch_size, learning_rate])                                   

                        
print('done')
print(datetime.datetime.now())

start making test settings...done
2023-09-22 14:21:26.894285


In [ ]:
from keras import metrics
from tensorflow.keras.callbacks import LearningRateScheduler
from keras.layers import Concatenate
from sklearn.metrics import mean_absolute_error, mean_squared_error

seed_everything(SEED)

def step_decay(epoch):
    start = 1e-3
    drop = 0.1
    epochs_drop = 10
    lr = start * (drop ** np.floor((epoch)/epochs_drop))
    return lr

# random search for hyperparameter
ntrial = ntest
train_errs, val_errs = [] ,[]
test_acc, test_roc, test_prc = [], [], []
#test_rmse, test_mae, test_auc = [], [], []
random_settings = []


for itrial in range(ntrial):
    # grid search
    # test_setting = test_settings[itrial]

    # random search
    print('random search {}/{}'.format(itrial, ntrial))
    
    # total conv layers of the model
    n_conv = random.choice([2,3,4]) 
    # test settings
    for i in range(n_conv):
        nfilt[i], kernels[i], strides[i] = random.choice(layer_settings)
    dense_node, dropout_cnn, dropout_fc, batch_size, learning_rate = random.choice(test_settings)
    
    if itrial < 0:
        continue
    

    # 이번 옵션에 대한 결과 디렉토리
    odir_f = f'batch{batch_size},'
    for i in range(n_conv):
        odir_f += f'layer{i+1}:c{nfilt[i]}filt{kernels[i]}str{strides[i]},'
    odir_f += f'1conv,dropout{dropout_cnn},dnodes{dense_node},dropout{dropout_fc}'#,lr{learning_rate}'
    random_settings.append(odir_f)
    
    odir = rootdir + '/' + odir_f
    if not os.path.exists(odir):
        os.mkdir(odir)

        
    # model validation (VGG)
    out_shape = x_train.shape[1]
    for i in range(n_conv):
        out_shape = out_shape / pool_size / strides[i]
    if out_shape < 1:
        print('non-valid structure')
        os.rmdir(odir)
        itrial -= 1
        continue
        
    #mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/GPU:0","/GPU:1","/GPU:2","/GPU:3"])
    #with mirrored_strategy.scope(): 
    
    with tf.device(f'/gpu:{GPU}'):
        # build a model
        inp = Input(shape=(x_train.shape[1],x_train.shape[2]), name="input_1")
        out = inp


        # VGC block
        for i in range(n_conv):
            out = Conv1D(filters=nfilt[i], kernel_size=kernels[i], strides=strides[i], padding='same', activation=None)(out)
            out = BatchNormalization()(out)
            out = Activation('relu')(out)
            out = MaxPooling1D(pool_size)(out)


        # globalpooling vs flattening vs 1x1 convolution
        #elif globalpool_opt == 'ave':
        #    out = GlobalAveragePooling1D()(out)
        out = Conv1D(filters=1, kernel_size=1)(out)
        out = Flatten() (out)

    
        # concat with X_feature
        inp_feat = Input(shape=(feat_train.shape[1]), name="input_2")
        out = Concatenate(axis=1)([out, inp_feat])

        if dense_node != 0:
            out = Dropout(dropout_cnn)(out)
            out = Dense(dense_node, activation='relu')(out)
        out = Dropout(dropout_fc)(out)
        out = Dense(1, activation='sigmoid')(out)


        model = Model(inputs=[inp, inp_feat], outputs=[out])
        lr_scheduler = LearningRateScheduler(step_decay, verbose=1)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=lr_scheduler, weight_decay=1e-4), metrics=['mean_absolute_error'])
        model.save_weights(f'{odir}/initial_weights.hdf5')


    # 4-fold cv
    kfold = KFold(nfold)
    tprs, aucs, prs = [], [], []
    test_rmse, test_mae = [], []
    f1_scores, thvals = [], []
    mean_fpr = np.linspace(0,1,100)
    mean_recall = np.linspace(0,1,100)

    switch = 0
    caseids_train = np.unique(c_train)
    for fold, (c_cv_trains_mask, c_cv_test_mask) in enumerate(kfold.split(caseids_train)):
        c_cv_trains = caseids_train[c_cv_trains_mask]

        cv_train_mask = np.isin(c_train, c_cv_trains)
        cv_val_mask = ~cv_train_mask

        X_train = x_train[cv_train_mask]
        X_val = x_train[cv_val_mask]

        Y_train = y_train[cv_train_mask]
        Y_val = y_train[cv_val_mask]

        Feat_train = feat_train[cv_train_mask]
        Feat_val = feat_train[cv_val_mask]

        
        # model 학습
        try:

            with tf.device("/CPU:0"):
                train_dataset = tf.data.Dataset.from_tensor_slices(({"input_1": X_train, "input_2": Feat_train}, Y_train)).shuffle(x_train.shape[0]).batch(batch_size)
                val_dataset = tf.data.Dataset.from_tensor_slices(({"input_1": X_val, "input_2": Feat_val}, Y_val)).batch(batch_size)
                test_dataset = tf.data.Dataset.from_tensor_slices(({"input_1": x_test, "input_2": feat_test}, y_test)).batch(batch_size)
       
            weightcache = f"{odir}/weights_{fold}.hdf5"
            #model = multi_gpu_model(model, gpus=4)
            #model.compile(loss='mean_squared_error', optimizer=Adam(lr=lr_scheduler, weight_decay=1e-4), metrics=['mean_absolute_error'])
            hist = model.fit(train_dataset, validation_data=val_dataset, epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                    callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')])

            model.load_weights(weightcache)
            y_pred = model.predict(test_dataset).flatten()

            # MAE 계산
            mae_val = mean_absolute_error(y_test, y_pred)
            test_mae.append(mae_val)


            print(f' ###{fold} fold : val mae {mae_val:.2f}###')
            tf.keras.backend.clear_session()
            model.load_weights(f'{odir}/initial_weights.hdf5')

        except Exception as e:
            print(e)
            switch = 1
            shutil.rmtree(odir)
            itrial -= 1

            break
    ###
    if switch:
        switch = 0
        tf.keras.backend.clear_session()
        continue

    # 

    
    mean_mae = np.mean(test_mae)
    std_mae = np.std(test_mae)

    max_idx = test_mae.index(min(test_mae))
    

    print(f'mae{mean_mae*SCALE_Y:.2f}+-{std_mae*SCALE_Y:.2f}')
    open(odir+"/model.json", "wt").write(model.to_json())

    os.rename(odir, rootdir+f'/mae{mean_mae*SCALE_Y:.2f}+-{std_mae*SCALE_Y:.2f}_max{max_idx}__{odir_f}')
    tf.keras.backend.clear_session()


random search 0/500


2023-09-22 14:23:51.352596: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-22 14:23:52.450645: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22325 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:81:00.0, compute capability: 8.6


Epoch 1/100


2023-09-22 14:23:59.311829: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8904
2023-09-22 14:24:00.367762: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-09-22 14:24:00.450508: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-09-22 14:24:00.453461: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f7d24009ec0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-22 14:24:00.453499: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA RTX A5000, Compute Capability 8.6
2023-09-22 14:24:00.462216: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-22 14:24:00.533351: I te

79/79 [==============================] - ETA: 0s - loss: 0.0532 - mean_absolute_error: 0.1815
Epoch 1: val_loss improved from inf to 0.02371, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers+feats_kernel(7~19)_age%100(sigmoid)_mse(decay)_4fold_test500/batch128,layer1:c64filt15str2,layer2:c16filt9str2,layer3:c32filt15str2,1conv,dropout0.5,dnodes64,dropout0.4/weights_0.hdf5
79/79 [==============================] - 10s 42ms/step - loss: 0.0532 - mean_absolute_error: 0.1815 - val_loss: 0.0237 - val_mean_absolute_error: 0.1192
Epoch 2/100
78/79 [============================>.] - ETA: 0s - loss: 0.0286 - mean_absolute_error: 0.1319
Epoch 2: val_loss improved from 0.02371 to 0.01563, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers+feats_kernel(7~19)_age%100(sigmoid)_mse(decay)_4fold_test500/batch128,layer1:c64filt15str2,layer2:c16filt9str2,layer3:c32filt15str2,1conv,dropout0.5,dnodes64,dropout0.4/weights_0.hdf5
79/79

...

Epoch 1/100
313/314 [============================>.] - ETA: 0s - loss: 0.0296 - mean_absolute_error: 0.1303
Epoch 1: val_loss improved from inf to 0.01648, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers+feats_kernel(7~19)_age%100(sigmoid)_mse(decay)_4fold_test500/batch32,layer1:c32filt19str2,layer2:c32filt7str2,layer3:c64filt7str4,1conv,dropout0,dnodes32,dropout0.3/weights_0.hdf5
314/314 [==============================] - 10s 17ms/step - loss: 0.0296 - mean_absolute_error: 0.1303 - val_loss: 0.0165 - val_mean_absolute_error: 0.1021
Epoch 2/100
313/314 [============================>.] - ETA: 0s - loss: 0.0168 - mean_absolute_error: 0.1014
Epoch 2: val_loss improved from 0.01648 to 0.01573, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers+feats_kernel(7~19)_age%100(sigmoid)_mse(decay)_4fold_test500/batch32,layer1:c32filt19str2,layer2:c32filt7str2,layer3:c64filt7str4,1conv,dropout0,dnodes32,dropout0.3/weights_0.

Epoch 1/100
314/314 [==============================] - ETA: 0s - loss: 0.0216 - mean_absolute_error: 0.1119
Epoch 1: val_loss improved from inf to 0.01658, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers+feats_kernel(7~19)_age%100(sigmoid)_mse(decay)_4fold_test500/batch32,layer1:c64filt9str4,layer2:c64filt9str2,layer3:c32filt19str2,1conv,dropout0,dnodes128,dropout0.1/weights_0.hdf5
314/314 [==============================] - 9s 18ms/step - loss: 0.0216 - mean_absolute_error: 0.1119 - val_loss: 0.0166 - val_mean_absolute_error: 0.0998
Epoch 2/100
313/314 [============================>.] - ETA: 0s - loss: 0.0135 - mean_absolute_error: 0.0904
Epoch 2: val_loss improved from 0.01658 to 0.01278, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers+feats_kernel(7~19)_age%100(sigmoid)_mse(decay)_4fold_test500/batch32,layer1:c64filt9str4,layer2:c64filt9str2,layer3:c32filt19str2,1conv,dropout0,dnodes128,dropout0.1/weights_0

Epoch 1/100
314/314 [==============================] - ETA: 0s - loss: 0.0290 - mean_absolute_error: 0.1326
Epoch 1: val_loss improved from inf to 0.01654, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers+feats_kernel(7~19)_age%100(sigmoid)_mse(decay)_4fold_test500/batch32,layer1:c16filt17str4,layer2:c16filt9str4,1conv,dropout0.2,dnodes16,dropout0.3/weights_0.hdf5
314/314 [==============================] - 7s 15ms/step - loss: 0.0290 - mean_absolute_error: 0.1326 - val_loss: 0.0165 - val_mean_absolute_error: 0.1011
Epoch 2/100
313/314 [============================>.] - ETA: 0s - loss: 0.0175 - mean_absolute_error: 0.1043
Epoch 2: val_loss improved from 0.01654 to 0.01415, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers+feats_kernel(7~19)_age%100(sigmoid)_mse(decay)_4fold_test500/batch32,layer1:c16filt17str4,layer2:c16filt9str4,1conv,dropout0.2,dnodes16,dropout0.3/weights_0.hdf5
314/314 [=======================

Epoch 1/100
79/79 [==============================] - ETA: 0s - loss: 0.0454 - mean_absolute_error: 0.1673
Epoch 1: val_loss improved from inf to 0.02230, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers+feats_kernel(7~19)_age%100(sigmoid)_mse(decay)_4fold_test500/batch128,layer1:c64filt17str2,layer2:c16filt17str4,layer3:c64filt13str4,1conv,dropout0.1,dnodes16,dropout0.2/weights_0.hdf5
79/79 [==============================] - 6s 42ms/step - loss: 0.0454 - mean_absolute_error: 0.1673 - val_loss: 0.0223 - val_mean_absolute_error: 0.1178
Epoch 2/100
77/79 [============================>.] - ETA: 0s - loss: 0.0241 - mean_absolute_error: 0.1215
Epoch 2: val_loss improved from 0.02230 to 0.02112, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers+feats_kernel(7~19)_age%100(sigmoid)_mse(decay)_4fold_test500/batch128,layer1:c64filt17str2,layer2:c16filt17str4,layer3:c64filt13str4,1conv,dropout0.1,dnodes16,dropout0.2/weights

Epoch 1/100
313/314 [============================>.] - ETA: 0s - loss: 0.0371 - mean_absolute_error: 0.1499
Epoch 1: val_loss improved from inf to 0.01724, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers+feats_kernel(7~19)_age%100(sigmoid)_mse(decay)_4fold_test500/batch32,layer1:c64filt19str2,layer2:c16filt13str4,layer3:c32filt13str4,1conv,dropout0.4,dnodes16,dropout0.1/weights_0.hdf5
314/314 [==============================] - 9s 17ms/step - loss: 0.0371 - mean_absolute_error: 0.1499 - val_loss: 0.0172 - val_mean_absolute_error: 0.1030
Epoch 2/100
314/314 [==============================] - ETA: 0s - loss: 0.0209 - mean_absolute_error: 0.1137
Epoch 2: val_loss improved from 0.01724 to 0.01427, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers+feats_kernel(7~19)_age%100(sigmoid)_mse(decay)_4fold_test500/batch32,layer1:c64filt19str2,layer2:c16filt13str4,layer3:c32filt13str4,1conv,dropout0.4,dnodes16,dropout0.1/wei

In [40]:
with tf.device("/CPU:0"):
    train_dataset = tf.data.Dataset.from_tensor_slices(({"input_1": X_train, "input_2": Feat_train}, Y_train)).shuffle(x_train.shape[0]).batch(batch_size)
    val_dataset = tf.data.Dataset.from_tensor_slices(({"input_1": X_val, "input_2": Feat_val}, Y_val)).batch(batch_size)
    test_dataset = tf.data.Dataset.from_tensor_slices(({"input_1": x_test, "input_2": feat_test}, y_test)).batch(batch_size)
    
    
weightcache = f"{odir}/weights_{fold}.hdf5"
#model = multi_gpu_model(model, gpus=4)
#model.compile(loss='mean_squared_error', optimizer=Adam(lr=lr_scheduler, weight_decay=1e-4), metrics=['mean_absolute_error'])
hist = model.fit(train_dataset, validation_data=val_dataset, epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                        callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                    EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')])

model.load_weights(weightcache)
y_pred = model.predict(test_dataset).flatten()

# MAE 계산
mae_val = mean_absolute_error(y_test, y_pred)
test_mae.append(mae_val)    
    

Epoch 1/100
164/164 [==============================] - ETA: 0s - loss: 0.0395 - mean_absolute_error: 0.1534
Epoch 1: val_loss improved from inf to 0.01552, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_female_train/CNN_4layers_feats_kernel(7~19)_age%100(sigmoid)_mae(decay)_4fold_test500/batch128,layer1:c64filt15str2,layer2:c16filt9str2,layer3:c32filt15str2,1conv,dropout0.5,dnodes64,dropout0.4/weights_0.hdf5
164/164 [==============================] - 9s 35ms/step - loss: 0.0395 - mean_absolute_error: 0.1534 - val_loss: 0.0155 - val_mean_absolute_error: 0.0981
Epoch 2/100
164/164 [==============================] - ETA: 0s - loss: 0.0200 - mean_absolute_error: 0.1100
Epoch 2: val_loss did not improve from 0.01552
164/164 [==============================] - 5s 28ms/step - loss: 0.0200 - mean_absolute_error: 0.1100 - val_loss: 0.0156 - val_mean_absolute_error: 0.0963
Epoch 3/100
162/164 [============================>.] - ETA: 0s - loss: 0.0157 - mean_absolute_error: 0.0980


In [25]:
with tf.device("/CPU:0"):
    train_dat1 = tf.data.Dataset.from_tensor_slices((X_train, Feat_train))
    train_dat2 = tf.data.Dataset.from_tensor_slices(Y_train)
    train_dataset = tf.data.Dataset.zip((train_dat1, train_dat2)).shuffle(x_train.shape[0]).batch(batch_size)

    val_dat1 = tf.data.Dataset.from_tensor_slices((X_val, Feat_val))
    val_dat2 = tf.data.Dataset.from_tensor_slices(Y_val)
    val_dataset = tf.data.Dataset.zip((val_dat1, val_dat2)).batch(batch_size)

    test_dat1 = tf.data.Dataset.from_tensor_slices((x_test, feat_test))
    test_dat2 = tf.data.Dataset.from_tensor_slices(y_test)
    test_dataset = tf.data.Dataset.zip((test_dat1, test_dat2)).batch(batch_size)

weightcache = f"{odir}/weights_{fold}.hdf5"
#model = multi_gpu_model(model, gpus=4)
#model.compile(loss='mean_squared_error', optimizer=Adam(lr=lr_scheduler, weight_decay=1e-4), metrics=['mean_absolute_error'])
hist = model.fit(train_dataset, validation_data=val_dataset, epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                        callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                    EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')])

model.load_weights(weightcache)
y_pred = model.predict(test_dataset).flatten()

# MAE 계산
mae_val = mean_absolute_error(y_test, y_pred)
test_mae.append(mae_val)

SyntaxError: invalid syntax (1580337902.py, line 2)

In [ ]:
from keras import metrics
from tensorflow.keras.callbacks import LearningRateScheduler
from keras.layers import Concatenate
from sklearn.metrics import mean_absolute_error, mean_squared_error

seed_everything(SEED)

# random search for hyperparameter
ntrial = ntest
train_errs, val_errs = [] ,[]
test_acc, test_roc, test_prc = [], [], []
#test_rmse, test_mae, test_auc = [], [], []
random_settings = []


for itrial in range(ntrial):
    # grid search
    # test_setting = test_settings[itrial]

    # random search
    print('random search {}/{}'.format(itrial, ntrial))
    
    # total conv layers of the model
    n_conv = random.choice([2,3,4]) 
    # test settings
    for i in range(n_conv):
        nfilt[i], kernels[i], strides[i] = random.choice(layer_settings)
    dense_node, dropout_cnn, dropout_fc, batch_size, learning_rate = random.choice(test_settings)
    
    if itrial < 0:
        continue
    

    # 이번 옵션에 대한 결과 디렉토리
    odir_f = f'batch{batch_size},'
    for i in range(n_conv):
        odir_f += f'layer{i+1}:c{nfilt[i]}filt{kernels[i]}str{strides[i]},'
    odir_f += f'1conv,dropout{dropout_cnn},dnodes{dense_node},dropout{dropout_fc}'#,lr{learning_rate}'
    random_settings.append(odir_f)
    
    odir = rootdir + '/' + odir_f
    if not os.path.exists(odir):
        os.mkdir(odir)

        
    # model validation (VGG)
    out_shape = x_train.shape[1]
    for i in range(n_conv):
        out_shape = out_shape / pool_size / strides[i]
    if out_shape < 1:
        print('non-valid structure')
        os.rmdir(odir)
        itrial -= 1
        continue
        
    #mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/GPU:0","/GPU:1","/GPU:2","/GPU:3"])
    #with mirrored_strategy.scope(): 
    
    with tf.device(f'/gpu:{GPU}'):
        # build a model
        inp = Input(shape=(x_train.shape[1],x_train.shape[2]))
        out = inp


        # VGC block
        for i in range(n_conv):
            out = Conv1D(filters=nfilt[i], kernel_size=kernels[i], strides=strides[i], padding='same', activation=None)(out)
            out = BatchNormalization()(out)
            out = Activation('relu')(out)
            out = MaxPooling1D(pool_size)(out)


        # globalpooling vs flattening vs 1x1 convolution
        #elif globalpool_opt == 'ave':
        #    out = GlobalAveragePooling1D()(out)
        out = Conv1D(filters=1, kernel_size=1)(out)
        out = Flatten() (out)

    
        # concat with X_feature
        inp_feat = Input(shape=(feat_train.shape[1]))
        out = Concatenate(axis=1)([out, inp_feat])

        if dense_node != 0:
            out = Dropout(dropout_cnn)(out)
            out = Dense(dense_node, activation='relu')(out)
        out = Dropout(dropout_fc)(out)
        out = Dense(1, activation='sigmoid')(out)


        model = Model(inputs=[inp, inp_feat], outputs=[out])
        model.save_weights(f'{odir}/initial_weights.hdf5')


        # 4-fold cv
        kfold = KFold(nfold)
        tprs, aucs, prs = [], [], []
        test_rmse, test_mae = [], []
        f1_scores, thvals = [], []
        mean_fpr = np.linspace(0,1,100)
        mean_recall = np.linspace(0,1,100)

        switch = 0
        caseids_train = np.unique(c_train)
        for fold, (c_cv_trains_mask, c_cv_test_mask) in enumerate(kfold.split(caseids_train)):
            c_cv_trains = caseids_train[c_cv_trains_mask]
            
            cv_train_mask = np.isin(c_train, c_cv_trains)
            cv_val_mask = ~cv_train_mask
            
            X_train = x_train[cv_train_mask]
            X_val = x_train[cv_val_mask]

            Y_train = y_train[cv_train_mask]
            Y_val = y_train[cv_val_mask]
            
            Feat_train = feat_train[cv_train_mask]
            Feat_val = feat_train[cv_val_mask]

            # model 학습
            try:
                # learning scheduler
                def step_decay(epoch):
                    start = 1e-3
                    drop = 0.1
                    epochs_drop = 10
                    lr = start * (drop ** np.floor((epoch)/epochs_drop))
                    return lr
                lr_scheduler = LearningRateScheduler(step_decay, verbose=1)
                
                weightcache = f"{odir}/weights_{fold}.hdf5"
                #model = multi_gpu_model(model, gpus=4)
                model.compile(loss='mean_squared_error', optimizer=Adam(lr=lr_scheduler, weight_decay=1e-4), metrics=['mean_absolute_error'])
                hist = model.fit([X_train, Feat_train], Y_train, validation_data=([X_val, Feat_val], Y_val), epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                        callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                    EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')])

                model.load_weights(weightcache)
                y_pred = model.predict([x_test, feat_test]).flatten()

                # MAE 계산
                mae_val = mean_absolute_error(y_test, y_pred)
                test_mae.append(mae_val)


                print(f' ###{fold} fold : val mae {mae_val:.2f}###')
                tf.keras.backend.clear_session()
                model.load_weights(f'{odir}/initial_weights.hdf5')

            except Exception as e:
                print(e)
                switch = 1
                shutil.rmtree(odir)
                itrial -= 1
                
                break
        ###
        if switch:
            switch = 0
            tf.keras.backend.clear_session()
            continue

    # 

    
    mean_mae = np.mean(test_mae)
    std_mae = np.std(test_mae)

    max_idx = test_mae.index(min(test_mae))
    

    print(f'mae{mean_mae*SCALE_Y:.2f}+-{std_mae*SCALE_Y:.2f}')
    open(odir+"/model.json", "wt").write(model.to_json())

    os.rename(odir, rootdir+f'/mae{mean_mae*SCALE_Y:.2f}+-{std_mae*SCALE_Y:.2f}_max{max_idx}__{odir_f}')
    tf.keras.backend.clear_session()


random search 0/500


Epoch 1/100
163/164 [============================>.] - ETA: 0s - loss: 0.0411 - mean_absolute_error: 0.1576
Epoch 1: val_loss improved from inf to 0.01682, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers_feats_kernel(7~19)_age%100(sigmoid)_mae(decay)_4fold_test500/batch128,layer1:c64filt15str2,layer2:c16filt9str2,layer3:c32filt15str2,1conv,dropout0.5,dnodes64,dropout0.4/weights_0.hdf5
164/164 [==============================] - 9s 39ms/step - loss: 0.0411 - mean_absolute_error: 0.1576 - val_loss: 0.0168 - val_mean_absolute_error: 0.1000
Epoch 2/100
162/164 [============================>.] - ETA: 0s - loss: 0.0202 - mean_absolute_error: 0.1108
Epoch 2: val_loss improved from 0.01682 to 0.01630, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers_feats_kernel(7~19)_age%100(sigmoid)_mae(decay)_4fold_test500/batch128,layer1:c64filt15str2,layer2:c16filt9str2,layer3:c32filt15str2,1conv,dropout0.5,dnodes64,dropout0.4/wei

Epoch 1/100
162/164 [============================>.] - ETA: 0s - loss: 0.0411 - mean_absolute_error: 0.1576
Epoch 1: val_loss improved from inf to 0.01461, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers_feats_kernel(7~19)_age%100(sigmoid)_mae(decay)_4fold_test500/batch128,layer1:c64filt15str2,layer2:c16filt9str2,layer3:c32filt15str2,1conv,dropout0.5,dnodes64,dropout0.4/weights_1.hdf5
164/164 [==============================] - 9s 39ms/step - loss: 0.0410 - mean_absolute_error: 0.1574 - val_loss: 0.0146 - val_mean_absolute_error: 0.0949
Epoch 2/100
162/164 [============================>.] - ETA: 0s - loss: 0.0204 - mean_absolute_error: 0.1114
Epoch 2: val_loss did not improve from 0.01461
164/164 [==============================] - 5s 29ms/step - loss: 0.0204 - mean_absolute_error: 0.1114 - val_loss: 0.0177 - val_mean_absolute_error: 0.1079
Epoch 3/100
163/164 [============================>.] - ETA: 0s - loss: 0.0161 - mean_absolute_error: 0.0993
Ep

Epoch 1/100
162/164 [============================>.] - ETA: 0s - loss: 0.0407 - mean_absolute_error: 0.1566
Epoch 1: val_loss improved from inf to 0.01673, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers_feats_kernel(7~19)_age%100(sigmoid)_mae(decay)_4fold_test500/batch128,layer1:c64filt15str2,layer2:c16filt9str2,layer3:c32filt15str2,1conv,dropout0.5,dnodes64,dropout0.4/weights_2.hdf5
164/164 [==============================] - 9s 35ms/step - loss: 0.0406 - mean_absolute_error: 0.1562 - val_loss: 0.0167 - val_mean_absolute_error: 0.0994
Epoch 2/100
162/164 [============================>.] - ETA: 0s - loss: 0.0206 - mean_absolute_error: 0.1119
Epoch 2: val_loss improved from 0.01673 to 0.01485, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN_4layers_feats_kernel(7~19)_age%100(sigmoid)_mae(decay)_4fold_test500/batch128,layer1:c64filt15str2,layer2:c16filt9str2,layer3:c32filt15str2,1conv,dropout0.5,dnodes64,dropout0.4/wei

2023-09-21 21:50:20.884728: W tensorflow/tsl/framework/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.56GiB (rounded to 1670880000)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-09-21 21:50:20.884793: I tensorflow/tsl/framework/bfc_allocator.cc:1034] BFCAllocator dump for GPU_0_bfc
2023-09-21 21:50:20.884815: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (256): 	Total Chunks: 264, Chunks in use: 262. 66.0KiB allocated for chunks. 65.5KiB in use in bin. 25.5KiB client-requested in use in bin.
2023-09-21 21:50:20.884827: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (512): 	Total Chunks: 5, Chunks in use: 0. 3.5KiB allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2023-09-21 21:50:20.884839: I tensorflow/tsl/framework/bfc_a

Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.
random search 2/500


2023-09-21 21:50:46.693234: W tensorflow/tsl/framework/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.66GiB (rounded to 5008320000)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-09-21 21:50:46.693302: I tensorflow/tsl/framework/bfc_allocator.cc:1034] BFCAllocator dump for GPU_0_bfc
2023-09-21 21:50:46.693321: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (256): 	Total Chunks: 302, Chunks in use: 302. 75.5KiB allocated for chunks. 75.5KiB in use in bin. 30.2KiB client-requested in use in bin.
2023-09-21 21:50:46.693332: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2023-09-21 21:50:46.693344: I tensorflow/tsl/framework/bfc_alloc

Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.
random search 3/500


2023-09-21 21:50:59.552829: W tensorflow/tsl/framework/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.66GiB (rounded to 5008320000)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-09-21 21:50:59.552915: I tensorflow/tsl/framework/bfc_allocator.cc:1034] BFCAllocator dump for GPU_0_bfc
2023-09-21 21:50:59.552934: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (256): 	Total Chunks: 330, Chunks in use: 330. 82.5KiB allocated for chunks. 82.5KiB in use in bin. 32.7KiB client-requested in use in bin.
2023-09-21 21:50:59.552946: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2023-09-21 21:50:59.552957: I tensorflow/tsl/framework/bfc_alloc

Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.
random search 4/500


In [16]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 5000, 12)]   0           []                               
                                                                                                  
 conv1d_4 (Conv1D)              (None, 2500, 64)     11584       ['input_3[0][0]']                
                                                                                                  
 batch_normalization_3 (BatchNo  (None, 2500, 64)    256         ['conv1d_4[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 activation_3 (Activation)      (None, 2500, 64)     0           ['batch_normalization_3[0][

## cnn-transformer

In [ ]:
from keras.models import Model
from keras.layers import Layer, LayerNormalization, Dense, Dropout, Conv1D, MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D, Input, concatenate, multiply, dot, MultiHeadAttention
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

def step_decay(epoch):
    start = 1e-3
    drop = 0.1
    epochs_drop = 10
    lr = start * (drop ** np.floor((epoch)/epochs_drop))
    return lr

random.seed(98)
SRATE = 500  # in hz
SEGLEN = 10 * SRATE  # samples
#BATCH_SIZE = 256
MAX_CASES = 500
nfold = 4

hyperparameters = {
    "nfilt" : [16, 32, 64, 128],
    'nhead' : [2, 4, 8],
    'kdim': [16, 64, 128, 256],
    "fnode" : [32, 64, 128],
    "clayer" : [2, 3, 4],
    "tlayer" : [3, 4, 5],
    "droprate" : [0.1, 0.2, 0.5],
    "filtsize" : [5, 7, 9, 11, 15, 19],
    'poolsize' : [2, 4, 5],
    'batch_size': [64, 128, 256]
}
keys, values = zip(*hyperparameters.items())
permutations_dicts = it.product(*values)
permutations_dicts = list(permutations_dicts)
random.shuffle(permutations_dicts)

for nfilt, nhead, kdim, fnode, clayer, tlayer, droprate, filtsize, poolsize, batch_size in permutations_dicts:
    test_start = time.time()

    tf.keras.backend.clear_session()
    
    rootdir = f'randomSearch/{hyper_path}/CNN+transformer_age%100(sigmoid)_loss(mae,nodecay)_{nfold}fold'
    odir_f = 'batch{}_cnn{}_filt{}_size{}_pool{}_do{}'.format(batch_size, clayer, nfilt, filtsize, poolsize, droprate)
    odir_f += '_tra{}_head{}_kdim{}_fnn{}'.format(tlayer, nhead, kdim, fnode)
    
    if not os.path.exists(rootdir):
        os.mkdir(rootdir)
    
    odir = rootdir+'/'+odir_f
    print("============================")
    print(odir)
    print("============================")

    # cnn-transformer
    # https://keras.io/examples/nlp/text_classification_with_transformer/
    strategy = tf.distribute.MirroredStrategy()
    with strategy.scope():
        out = inp = Input(shape=(x_train.shape[1], x_train.shape[2]))

        # out = Conv1D(filters=kdim, kernel_size=filtsize, padding='same')(out)

        # conv 여러층    
        for i in range(clayer):
            out = Conv1D(filters=nfilt, kernel_size=filtsize, padding='same', activation=None)(out)
            out = BatchNormalization()(out)
            out = Activation('relu')(out)
            out = MaxPooling1D(poolsize, padding='same')(out)
        out = Dense(kdim)(out)  # 마지막 차원이 nfilt 인데 kdim 으로 바꿔야 transformer block을 쌓을 수 있다.
        for i in range(tlayer):  # transformer
            attn_output = MultiHeadAttention(num_heads=nhead, key_dim=kdim, attention_axes=[1,])(out, out)
            attn_output = Dropout(droprate)(attn_output)
            out1 = LayerNormalization(epsilon=1e-6)(out + attn_output)  # sum and norm
            ffn_output = tf.keras.Sequential([Dense(fnode, activation="relu"), Dense(kdim)])(out1)
            out2 = Dropout(droprate)(ffn_output)
            out = LayerNormalization(epsilon=1e-6)(out1 + out2)  # sum and norm
        out = GlobalMaxPooling1D()(out)

        if droprate:
            out = Dropout(droprate)(out)
        out = Dense(fnode)(out)
        if droprate:
            out = Dropout(droprate)(out)
        out = Dense(1, activation='sigmoid')(out)

        if not os.path.exists(odir):
            os.mkdir(odir)

        #cache_path = odir + "/initial_weights.hdf5"
        model = Model(inputs=[inp], outputs=[out])
        lr_scheduler = LearningRateScheduler(step_decay, verbose=1)
        model.compile(loss='mae', optimizer=Adam(lr=lr_scheduler, weight_decay=None), metrics=['mean_squared_error'])
        model.save_weights(f'{odir}/initial_weights.hdf5')


    # 4-fold cv
    kfold = KFold(nfold)
    tprs, aucs, prs = [], [], []
    test_rmse, test_mae = [], []
    f1_scores, thvals = [], []
    mean_fpr = np.linspace(0,1,100)
    mean_recall = np.linspace(0,1,100)

    switch = 0
    caseids_train = np.unique(c_train)
    for fold, (c_cv_trains_mask, c_cv_test_mask) in enumerate(kfold.split(caseids_train)):
        c_cv_trains = caseids_train[c_cv_trains_mask]

        cv_train_mask = np.isin(c_train, c_cv_trains)
        cv_val_mask = ~cv_train_mask

        X_train = x_train[cv_train_mask]
        X_val = x_train[cv_val_mask]

        Y_train = y_train[cv_train_mask]
        Y_val = y_train[cv_val_mask]


        # model 학습
        try:
            # learning scheduler
            

            weightcache = f"{odir}/weights_{fold}.hdf5"
            #model = multi_gpu_model(model, gpus=4)
            #model.compile(loss='mae', optimizer=Adam(lr=lr_scheduler, weight_decay=None), metrics=['mean_squared_error'])
            hist = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                    callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')])

            model.load_weights(weightcache)
            y_pred = model.predict(x_test).flatten()

            # MAE 계산
            model_err = metrics.MeanAbsoluteError()
            model_err.update_state(y_test, y_pred)
            mae_val = model_err.result().numpy()
            test_mae.append(mae_val)

            # RMSE 계산
            model_err = metrics.RootMeanSquaredError() 
            model_err.update_state(y_test, y_pred)
            rmse_val = model_err.result().numpy()
            test_rmse.append(rmse_val)


            print(f' ###{fold} fold : val mae {mae_val:.2f}, val rmse {rmse_val:.2f}###')
            tf.keras.backend.clear_session()
            model.load_weights(f'{odir}/initial_weights.hdf5')

        except Exception as e:
            print(e)
            switch = 1
            shutil.rmtree(odir)
            break
    ###
    if switch:
        switch = 0
        tf.keras.backend.clear_session()
        continue


    # RMSE 계산
    mean_rmse = np.mean(test_rmse)
    std_rmse = np.std(test_rmse)

    mean_mae = np.mean(test_mae)
    std_mae = np.std(test_mae)

    max_idx = test_mae.index(min(test_mae))


    print(f'mae{mean_mae*100:.2f}+-{std_mae*100:.2f}_rmse{mean_rmse*100:.2f}+-{std_rmse*100:.2f}')
    open(odir+"/model.json", "wt").write(model.to_json())

    os.rename(odir, rootdir+f'/mae{mean_mae*100:.2f}+-{std_mae*100:.2f}_rmse{mean_rmse*100:.2f}+-{std_rmse*100:.2f}_max{max_idx}__{odir_f}')
    tf.keras.backend.clear_session()


randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN+transformer_age%100(sigmoid)_loss(mae,nodecay)_4fold/batch64_cnn4_filt128_size9_pool5_do0.2_tra5_head8_kdim64_fnn64
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


Epoch 1/100
INFO:tensorflow:batch_all_reduce: 102 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 102 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 102 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 102 all-reduces with algorithm = nccl, num_packs = 1
2023-09-21 22:54:23.476576: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8904
2023-09-21 22:54:24.865965: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8904
2023-09-21 22:54:24.932444: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-09-21 22:54:25.809111: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


## CNN transformer + feat concat

In [ ]:
from keras.models import Model
from keras.layers import Layer, LayerNormalization, Dense, Dropout, Conv1D, MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D, Input, concatenate, multiply, dot, MultiHeadAttention
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

# learning scheduler
def step_decay(epoch):
    start = 1e-3
    drop = 0.1
    epochs_drop = 10
    lr = start * (drop ** np.floor((epoch)/epochs_drop))
    return lr

random.seed(98)
SRATE = 500  # in hz
SEGLEN = 10 * SRATE  # samples
#BATCH_SIZE = 256
MAX_CASES = 500
nfold = 4

hyperparameters = {
    "nfilt" : [16, 32, 64, 128],
    "stride": [2,4,5],
    'nhead' : [2, 4, 8],
    'kdim': [16, 64, 128, 256],
    "fnode" : [32, 64, 128],
    "clayer" : [2, 3, 4],
    "tlayer" : [3, 4, 5],
    "droprate" : [0.1, 0.2, 0.5],
    "filtsize" : [7, 9, 11, 15, 19],
    'poolsize' : [2, 4, 5],
    'batch_size': [64, 128, 256]
}
keys, values = zip(*hyperparameters.items())
permutations_dicts = it.product(*values)
permutations_dicts = list(permutations_dicts)
random.shuffle(permutations_dicts)
for nfilt, stride, nhead, kdim, fnode, clayer, tlayer, droprate, filtsize, poolsize, batch_size in permutations_dicts:
    test_start = time.time()

    tf.keras.backend.clear_session()
    
    rootdir = f'randomSearch/{hyper_path}/CNN+transformer+feats_age%{SCALE_Y}(sigmoid)_loss(mae)_{nfold}fold'
    odir_f = 'batch{}_cnn{}_filt{}_stride{}_size{}_pool{}_do{}'.format(batch_size, clayer, nfilt, stride, filtsize, poolsize, droprate)
    odir_f += '_tra{}_head{}_kdim{}_fnn{}'.format(tlayer, nhead, kdim, fnode)
    
    if not os.path.exists(rootdir):
        os.mkdir(rootdir)
    
    odir = rootdir+'/'+odir_f
    print("============================")
    print(odir)
    print("============================")

    # cnn-transformer
    # https://keras.io/examples/nlp/text_classification_with_transformer/
    #strategy = tf.distribute.MirroredStrategy()
    #with strategy.scope():
    with tf.device(f'/gpu:{GPU}'):
        out = inp = Input(shape=(x_train.shape[1], x_train.shape[2]))

        # out = Conv1D(filters=kdim, kernel_size=filtsize, padding='same')(out)

        # conv 여러층    
        for i in range(clayer):
            out = Conv1D(filters=nfilt, kernel_size=filtsize, strides=stride, padding='same', activation=None)(out)
            out = BatchNormalization()(out)
            out = Activation('relu')(out)
            out = MaxPooling1D(poolsize, padding='same')(out)
        out = Dense(kdim)(out)  # 마지막 차원이 nfilt 인데 kdim 으로 바꿔야 transformer block을 쌓을 수 있다.
        for i in range(tlayer):  # transformer
            attn_output = MultiHeadAttention(num_heads=nhead, key_dim=kdim, attention_axes=[1,])(out, out)
            attn_output = Dropout(droprate)(attn_output)
            out1 = LayerNormalization(epsilon=1e-6)(out + attn_output)  # sum and norm
            ffn_output = tf.keras.Sequential([Dense(fnode, activation="relu"), Dense(kdim)])(out1)
            out2 = Dropout(droprate)(ffn_output)
            out = LayerNormalization(epsilon=1e-6)(out1 + out2)  # sum and norm
        out = GlobalMaxPooling1D()(out)

        # concat with X_feature
        inp_feat = Input(shape=(feat_train.shape[1]), name="input_2")
        out = Concatenate(axis=1)([out, inp_feat])
        
        if droprate:
            out = Dropout(droprate)(out)
        out = Dense(fnode)(out)
        if droprate:
            out = Dropout(droprate)(out)
        out = Dense(1, activation='sigmoid')(out)

        if not os.path.exists(odir):
            os.mkdir(odir)

        #cache_path = odir + "/initial_weights.hdf5"
        model = Model(inputs=[inp, inp_feat], outputs=[out])
        lr_scheduler = LearningRateScheduler(step_decay, verbose=1)
        model.compile(loss='mae', optimizer=Adam(lr=lr_scheduler, weight_decay=None), metrics=['mean_squared_error'])
        model.save_weights(f'{odir}/initial_weights.hdf5')
    
    
    # 4-fold cv
    kfold = KFold(nfold)
    tprs, aucs, prs = [], [], []
    test_rmse, test_mae = [], []
    f1_scores, thvals = [], []
    mean_fpr = np.linspace(0,1,100)
    mean_recall = np.linspace(0,1,100)

    switch = 0
    caseids_train = np.unique(c_train)
    for fold, (c_cv_trains_mask, c_cv_test_mask) in enumerate(kfold.split(caseids_train)):
        c_cv_trains = caseids_train[c_cv_trains_mask]

        cv_train_mask = np.isin(c_train, c_cv_trains)
        cv_val_mask = ~cv_train_mask

        X_train = x_train[cv_train_mask]
        X_val = x_train[cv_val_mask]

        Y_train = y_train[cv_train_mask]
        Y_val = y_train[cv_val_mask]
        
        Feat_train = feat_train[cv_train_mask]
        Feat_val = feat_train[cv_val_mask]

        # model 학습
        try:
            # loading dataset
            with tf.device("/CPU:0"):
                train_dataset = tf.data.Dataset.from_tensor_slices(({"input_1": X_train, "input_2": Feat_train}, Y_train)).shuffle(x_train.shape[0]).batch(batch_size)
                val_dataset = tf.data.Dataset.from_tensor_slices(({"input_1": X_val, "input_2": Feat_val}, Y_val)).batch(batch_size)
                test_dataset = tf.data.Dataset.from_tensor_slices(({"input_1": x_test, "input_2": feat_test}, y_test)).batch(batch_size)
                
            weightcache = f"{odir}/weights_{fold}.hdf5"
            #model = multi_gpu_model(model, gpus=4)
            #model.compile(loss='mae', optimizer=Adam(lr=lr_scheduler, weight_decay=None), metrics=['mean_squared_error'])
            hist = model.fit(train_dataset, validation_data=val_dataset, epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                    callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')])

            model.load_weights(weightcache)
            y_pred = model.predict(test_dataset).flatten()

            # MAE 계산
            model_err = metrics.MeanAbsoluteError()
            model_err.update_state(y_test, y_pred)
            mae_val = model_err.result().numpy()
            test_mae.append(mae_val)



            print(f' ###{fold} fold : val mae {mae_val:.2f}###')
            tf.keras.backend.clear_session()
            model.load_weights(f'{odir}/initial_weights.hdf5')

        except Exception as e:
            print(e)
            switch = 1
            shutil.rmtree(odir)
            break
    ###
    if switch:
        switch = 0
        tf.keras.backend.clear_session()
        continue


    mean_mae = np.mean(test_mae)
    std_mae = np.std(test_mae)

    max_idx = test_mae.index(min(test_mae))


    print(f'mae{mean_mae*SCALE_Y:.2f}+-{std_mae*SCALE_Y:.2f}')
    open(odir+"/model.json", "wt").write(model.to_json())

    os.rename(odir, rootdir+f'/mae{mean_mae*SCALE_Y:.2f}+-{std_mae*SCALE_Y:.2f}_max{max_idx}__{odir_f}')
    tf.keras.backend.clear_session()


randomSearch/SRATE500_ECG-bandpass_z-norm_adult_female_train/CNN+transformer+feats_age%100(sigmoid)_loss(mae,nodecay)_4fold/batch256_cnn2_filt128_stride5_size11_pool4_do0.5_tra3_head4_kdim64_fnn64


2023-09-21 23:22:44.808849: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-21 23:22:46.258468: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21957 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:81:00.0, compute capability: 8.6
2023-09-21 23:22:46.259126: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 21956 MB memory:  -> device: 1, name: NVIDIA RTX A5000, pci bus id: 0000:82:00.0, compute capability: 8.6


Epoch 1/100


2023-09-21 23:22:58.140346: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8904
2023-09-21 23:22:59.167260: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-09-21 23:22:59.238780: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-09-21 23:22:59.248267: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x26809f70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-21 23:22:59.248320: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA RTX A5000, Compute Capability 8.6
2023-09-21 23:22:59.248332: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (1): NVIDIA RTX A5000, Compute Capability 8.6
2023-09-21 23:22:59.256344: I tensorflow/compiler/mlir/tensorflow/utils/d

82/82 [==============================] - ETA: 0s - loss: 0.2738 - mean_squared_error: 0.1090
Epoch 1: val_loss improved from inf to 0.15560, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_female_train/CNN+transformer+feats_age%100(sigmoid)_loss(mae,nodecay)_4fold/batch256_cnn2_filt128_stride5_size11_pool4_do0.5_tra3_head4_kdim64_fnn64/weights_0.hdf5
82/82 [==============================] - 18s 74ms/step - loss: 0.2738 - mean_squared_error: 0.1090 - val_loss: 0.1556 - val_mean_squared_error: 0.0429
Epoch 2/100
81/82 [============================>.] - ETA: 0s - loss: 0.1609 - mean_squared_error: 0.0414
Epoch 2: val_loss improved from 0.15560 to 0.13258, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_female_train/CNN+transformer+feats_age%100(sigmoid)_loss(mae,nodecay)_4fold/batch256_cnn2_filt128_stride5_size11_pool4_do0.5_tra3_head4_kdim64_fnn64/weights_0.hdf5
82/82 [==============================] - 4s 52ms/step - loss: 0.1608 - mean_squared_error: 0.04

2023-09-21 23:24:57.416091: W tensorflow/tsl/framework/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.55GiB (rounded to 1669440000)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-09-21 23:24:57.416173: I tensorflow/tsl/framework/bfc_allocator.cc:1034] BFCAllocator dump for GPU_0_bfc
2023-09-21 23:24:57.416190: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (256): 	Total Chunks: 135, Chunks in use: 133. 33.8KiB allocated for chunks. 33.2KiB in use in bin. 18.3KiB client-requested in use in bin.
2023-09-21 23:24:57.416203: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (512): 	Total Chunks: 23, Chunks in use: 22. 12.0KiB allocated for chunks. 11.5KiB in use in bin. 11.0KiB client-requested in use in bin.
2023-09-21 23:24:57.416214: I tensorflow/tsl/fr

Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.
randomSearch/SRATE500_ECG-bandpass_z-norm_adult_female_train/CNN+transformer+feats_age%100(sigmoid)_loss(mae,nodecay)_4fold/batch128_cnn4_filt16_stride2_size11_pool5_do0.2_tra3_head8_kdim128_fnn32


Epoch 1/100
164/164 [==============================] - ETA: 0s - loss: 0.2120 - mean_squared_error: 0.0726
Epoch 1: val_loss improved from inf to 0.15259, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_female_train/CNN+transformer+feats_age%100(sigmoid)_loss(mae,nodecay)_4fold/batch128_cnn4_filt16_stride2_size11_pool5_do0.2_tra3_head8_kdim128_fnn32/weights_0.hdf5
164/164 [==============================] - 18s 45ms/step - loss: 0.2120 - mean_squared_error: 0.0726 - val_loss: 0.1526 - val_mean_squared_error: 0.0402
Epoch 2/100
164/164 [==============================] - ETA: 0s - loss: 0.1200 - mean_squared_error: 0.0235
Epoch 2: val_loss improved from 0.15259 to 0.10390, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_female_train/CNN+transformer+feats_age%100(sigmoid)_loss(mae,nodecay)_4fold/batch128_cnn4_filt16_stride2_size11_pool5_do0.2_tra3_head8_kdim128_fnn32/weights_0.hdf5
164/164 [==============================] - 6s 34ms/step - loss: 0.1200 - mean

## ISIBrno-AIMT Res net + multi-head attention [대회링크](https://moody-challenge.physionet.org/2021/results/)

In [ ]:
with tf.device(f'/gpu:{GPU}'):
    # build a model
    inp = Input(shape=(x_train.shape[1],x_train.shape[2]))
    out = inp


    # VGC block
    for i in range(n_conv):
        out = Conv1D(filters=nfilt[i], kernel_size=kernels[i], strides=strides[i], padding='same', activation='relu')(out)
        out = BatchNormalization()(out)
        out = MaxPooling1D(pool_size)(out)


    # globalpooling vs flattening vs 1x1 convolution
    #elif globalpool_opt == 'ave':
    #    out = GlobalAveragePooling1D()(out)
    out = Conv1D(filters=1, kernel_size=1)(out)
    out = Flatten() (out)
    
    
    # concat with X_feature
    inp_feat = Input(shape=(x_feature.shape[1]))
    out = Concatenate(axis=1)([out, inp_feat])


    if dense_node != 0:
        out = Dropout(dropout_cnn)(out)
        out = Dense(dense_node, activation='relu')(out)
    out = Dropout(dropout_fc)(out)
    out = Dense(1, activation='linear')(out)


    model = Model(inputs=[inp, inp_feat], outputs=[out])
    model.save_weights(f'{odir}/initial_weights.hdf5')


    # 4-fold cv
    kfold = KFold(nfold)
    tprs, aucs, prs = [], [], []
    test_rmse, test_mae = [], []
    f1_scores, thvals = [], []
    mean_fpr = np.linspace(0,1,100)
    mean_recall = np.linspace(0,1,100)

    switch = 0
    caseids_train = np.unique(c_train)
    for fold, (c_cv_trains_mask, c_cv_test_mask) in enumerate(kfold.split(caseids_train)):
        c_cv_trains = caseids_train[c_cv_trains_mask]
        random.shuffle(c_cv_trains)

        ncv = int(len(c_cv_trains)*0.8)
        c_cv_train = c_cv_trains[:ncv]
        c_cv_val = c_cv_trains[ncv:]

        cv_train_mask = np.isin(c_train, c_cv_train)
        cv_val_mask = np.isin(c_train, c_cv_val)
        cv_test_mask = np.isin(c_train, caseids_train[c_cv_test_mask])

        X_train = x_train[cv_train_mask]
        X_val = x_train[cv_val_mask]
        X_test = x_train[cv_test_mask] 

        Y_train = y_train[cv_train_mask]
        Y_val = y_train[cv_val_mask]
        Y_test = y_train[cv_test_mask]
        


        # model 학습
        try:
            weightcache = f"{odir}/weights_{fold}.hdf5"
            #model = multi_gpu_model(model, gpus=4)
            model.compile(loss='mean_squared_error', optimizer=Adam(lr=learning_rate), metrics=['mean_absolute_error'])
            hist = model.fit([X_train, , Y_train, validation_data=(X_val, Y_val), epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                    callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')])

            model.load_weights(weightcache)
            y_pred = model.predict(X_test).flatten()

            # MAE 계산
            model_err = metrics.MeanAbsoluteError()
            model_err.update_state(Y_test, y_pred)
            mae_val = model_err.result().numpy()
            test_mae.append(mae_val)

            # RMSE 계산
            model_err = metrics.RootMeanSquaredError() 
            model_err.update_state(Y_test, y_pred)
            rmse_val = model_err.result().numpy()
            test_rmse.append(rmse_val)


            print(f' ###{fold} fold : val mae {mae_val:.2f}, val rmse {rmse_val:.2f}###')
            tf.keras.backend.clear_session()
            model.load_weights(f'{odir}/initial_weights.hdf5')

# BRNO team

In [3]:
SRATE = 500
ECG_FILT = 'bandpass'
TRAIN = 'train'
ADULT = 'adult'
NORM = 'z-norm'
GENDER = 'male'

SMALL  = ''


hyper_path = f'SRATE{SRATE}_ECG-{ECG_FILT}_{NORM}_{ADULT}_{GENDER}_{TRAIN}'
FILENAME = f'dataset/{ADULT}_train_{GENDER}{SMALL}.npz'
with np.load(open(FILENAME, 'rb'), allow_pickle=True) as f:
    caseid = f['caseid']
    x_feature, x_signal, y = f['X_feature'], f['X_signal'], f['y']

    
x_signal = np.delete(x_signal, 5559, axis=0)
x_feature = np.delete(x_feature, 5559, axis=0)
y = np.delete(y, 5559, axis=0)
caseid = np.delete(caseid, 5559, axis=0)        
    
x_train = x_signal
print(caseid.shape, x_feature.shape, x_signal.shape, y.shape)

(13390,) (13390, 55) (13390, 5000, 12) (13390,)


In [2]:
SRATE = 500
ECG_FILT = 'bandpass'
TRAIN = 'train'
ADULT = 'adult'
NORM = 'z-norm'
GENDER = 'male'

hyper_path = f'SRATE{SRATE}_ECG-{ECG_FILT}_{NORM}_{ADULT}_{TRAIN}'
input_path = f'dataset/{hyper_path}/signal+features/'

with np.load(input_path+'x.npz', allow_pickle=True) as f:
    x_train = f['x_train']
    x_test = f['x_test']
     
with np.load(input_path+'feats.npz', allow_pickle=True) as f:
    feat_train = f['feat_train']
    feat_test = f['feat_test']
        
with np.load(input_path+'y.npz', allow_pickle=True) as f:
    y_train = f['y_train']
    y_test = f['y_test']
    
with np.load(input_path+'c.npz', allow_pickle=True) as f:
    c_train = f['c_train']
    c_test = f['c_test']  
    
with np.load(input_path+'g.npz', allow_pickle=True) as f:
    g_train = f['g_train']
    g_test = f['g_test']        
    
print(f'x train shape: {x_train.shape}')
print(f'feat train shape: {feat_train.shape}')
print(f'x test shape: {x_test.shape}')
print(f'feat test shape: {feat_test.shape}')

hyper_path = f'SRATE{SRATE}_ECG-{ECG_FILT}_{NORM}_{ADULT}_{GENDER}_{TRAIN}'

NameError: name 'np' is not defined

In [4]:
SCALE_Y = 100
y_train = y_train / SCALE_Y
y_test = y_test / SCALE_Y

In [5]:
nfold, ntest = 3, 500
rootdir = f"randomSearch/{hyper_path}/BRNO_4layers_age%{SCALE_Y}(linear)_mse(decay)_{nfold}fold_test{ntest}"

if not os.path.exists(f"randomSearch/{hyper_path}"):
    os.mkdir(f"randomSearch/{hyper_path}")

if not os.path.exists(rootdir):
    os.mkdir(rootdir)



# hyperparamters pool
filt_opts = [32, 64, 128, 256] # num of filters(kernel)
stride_opts = [2,4]  # other opts: stride = (kernel-1)/2
kernel_opts = range(3,17,2) # kernel size

head_opts = [2, 3, 4, 8]
headsize_opts = [64, 128, 256]
dropout_opts  = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5] # dropout rate
dense_opts = [8, 16, 32, 64]
BATCH_SIZE = [32, 64, 128, 256, 512]
lr_opts = [0.001, 0.002, 0.0005]

# test settings
print('start making test settings...', end='', flush=True)

n_conv_max = 4
nfilt   = [0 for _ in range(n_conv_max)]
kernels = [0 for _ in range(n_conv_max)]
strides = [0 for _ in range(n_conv_max)]

layer_settings = []
for nfilter in filt_opts:
    for kernel in kernel_opts:
        for stride in stride_opts:
            #layer_settings.append([nfilter, kernel, int((kernel-1)/2)])
            layer_settings.append([nfilter, kernel, stride])

test_settings = []
for nhead in head_opts:
    for headsize in headsize_opts:
        for dense_node in dense_opts:
            for dropout_mha in dropout_opts:
                for dropout_fc in dropout_opts:
                    for batch_size in BATCH_SIZE:
                        for learning_rate in lr_opts:
                            test_settings.append([nhead, headsize, dense_node, dropout_mha, dropout_fc, batch_size, learning_rate])

print('done')
print(datetime.datetime.now())

start making test settings...done
2023-09-21 15:48:31.754758


In [ ]:
from keras.layers import MultiHeadAttention, GlobalMaxPool1D, Concatenate, LeakyReLU
seed_everything(SEED)

ntrial = ntest
train_errs, val_errs = [] ,[]
random_settings = []

for itrial in range(ntrial):
    # random search
    print('random search {}/{}'.format(itrial, ntrial))

    # total conv layers of the model
    n_conv = random.choice(range(2, n_conv_max + 1))

    # test settings
    for i in range(n_conv):
        nfilt[i], kernels[i], strides[i] = random.choice(layer_settings)
    nhead, head_size, dense_node, dropout_mha, dropout_fc, batch_size, learning_rate = random.choice(test_settings)

    
    # output directory
    odir_f = f'batch{batch_size},'
    for i in range(n_conv):
        odir_f += f'layer{i+1}:c{nfilt[i]}filt{kernels[i]}str{strides[i]},'
    odir_f += f'mha:nhead{nhead}hsize{headsize},dropout{dropout_mha},'
    odir_f += f'dnodes{dense_node},dropout{dropout_fc},lr{learning_rate}'
    random_settings.append(odir_f)

    odir = rootdir + '/' + odir_f
    if not os.path.exists(odir):
        os.mkdir(odir)

    # model validation
    out_shape = x_train.shape[1]
    out_shape = out_shape / strides[0]
    for i in range(1, n_conv):
        # residual block에서는 2번씩 거쳐서 stride를 2번 나눠줌
        out_shape = out_shape / strides[i] / strides[i]

    if out_shape < 1:
        print('non-valid structure')
        os.rmdir(odir)
        itrial -= 1
        continue

    #gpus = tf.config.list_logical_devices('GPU')
    #strategy = tf.distribute.MirroredStrategy(gpus)   
    if True:
    #with strategy.scope():
    #with tf.device(f'/gpu:{GPU}'):
        # ==================================================
        # BUILD MODEL
        # ==================================================
        # input layer
        inp_sig = Input(shape=(x_train.shape[1], x_train.shape[2]))
        out = inp_sig

        # first conv1d layer
        out = Conv1D(filters=nfilt[0], kernel_size=kernels[0], strides=strides[0], padding='same', activation=None)(out)
        out = BatchNormalization()(out)
        out = LeakyReLU(alpha=0.01)(out)

        # stacked residual blocks
        for i in range(1, n_conv):
            res = out

            out = Conv1D(filters=nfilt[i], kernel_size=kernels[i], strides=strides[i], padding='same', activation=None)(out)
            out = BatchNormalization()(out)
            out = LeakyReLU(alpha=0.01)(out)
            out = Conv1D(filters=nfilt[i], kernel_size=kernels[i], strides=strides[i], padding='same', activation=None)(out)
            out = BatchNormalization()(out)
            out = LeakyReLU(alpha=0.01)(out)
            
            res = AveragePooling1D(pool_size=strides[i], padding='same')(res)
            res = Conv1D(filters=nfilt[i], kernel_size=kernels[i], strides=strides[i], padding='same', activation=None)(res)

            out = res + out

        # multi-head attention
        out = Dropout(dropout_mha)(out)
        out = MultiHeadAttention(num_heads=nhead, key_dim=headsize)(out, out)
        out = GlobalMaxPool1D()(out)

        # concat with X_feature
        inp_feat = Input(shape=(feat_train.shape[1]))
        out = Concatenate(axis=1)([out, inp_feat])

        # final dense layer
        out = Dropout(dropout_fc)(out)
        out = Dense(dense_node, activation=None)(out)
        out = BatchNormalization()(out)
        out = LeakyReLU(alpha=0.01)(out)
        
        out = Dropout(dropout_fc)(out)
        out = Dense(dense_node, activation=None)(out)
        out = LeakyReLU(alpha=0.01)(out)
        out = Dense(1, activation='linear')(out)
        
        # build model
        model = Model(inputs=[inp_sig, inp_feat], outputs=[out])
        model.save_weights(f'{odir}/initial_weights.hdf5')

        # ==================================================
        # K FOLD CROSS VALIDATION
        # ==================================================
        kfold = KFold(n_splits=nfold)
        tprs, aucs, prs = [], [], []
        test_rmse, test_mae = [], []
        f1_scores, thvals = [], []
        mean_fpr = np.linspace(0,1,100)
        mean_recall = np.linspace(0,1,100)

        switch = 0
        caseids_train = np.unique(c_train)
        for fold, (c_cv_trains_mask, c_cv_test_mask) in enumerate(kfold.split(caseids_train)):
            c_cv_trains = caseids_train[c_cv_trains_mask]
            
            cv_train_mask = np.isin(c_train, c_cv_trains)
            cv_val_mask = ~cv_train_mask
            
            X_train = x_train[cv_train_mask]
            X_val = x_train[cv_val_mask]

            Y_train = y_train[cv_train_mask]
            Y_val = y_train[cv_val_mask]
            
            Feat_train = feat_train[cv_train_mask]
            Feat_val = feat_train[cv_val_mask]


            # ==================================================
            # TRAIN MODEL
            # ==================================================
            try:
                weightcache = f"{odir}/weights_{fold}.hdf5"
                model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=learning_rate), metrics=['mean_absolute_error'])
                hist = model.fit([X_train, Feat_train], Y_train, validation_data=([X_val, Feat_val], Y_val), epochs=100, batch_size=batch_size,
                                 callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                 EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')])

                print('next step')
                model.load_weights(weightcache)
                y_pred = model.predict([x_test, feat_test]).flatten()

                print('mae cal')
                # MAE 계산
                
                model_err = metrics.MeanAbsoluteError()
                model_err.update_state(y_test, y_pred)
                mae_val = model_err.result().numpy()
                test_mae.append(mae_val)


                print(f' ###{fold} fold : val mae {mae_val:.2f}###')
                tf.keras.backend.clear_session()
                model.load_weights(f'{odir}/initial_weights.hdf5')

            except Exception as e:
                print(e)
                switch = 1
                shutil.rmtree(odir)
                itrial -= 1
                
                break
        ###
        if switch:
            switch = 0
            tf.keras.backend.clear_session()
            continue

    # RMSE 계산
    mean_rmse = np.mean(test_rmse)
    std_rmse = np.std(test_rmse)

    mean_mae = np.mean(test_mae)
    std_mae = np.std(test_mae)

    max_idx = test_mae.index(min(test_mae))


    print(f'mae{mean_mae*SCALE_Y:.2f}+-{std_mae*SCALE_Y:.2f}')
    open(odir+"/model.json", "wt").write(model.to_json())

    os.rename(odir, rootdir+f'/mae{mean_mae*SCALE_Y:.2f}+-{std_mae*SCALE_Y:.2f}_max{max_idx}__{odir_f}')
    tf.keras.backend.clear_session()

random search 0/500


2023-09-21 15:48:31.844634: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-21 15:48:32.298148: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22325 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:81:00.0, compute capability: 8.6


Epoch 1/100


2023-09-21 15:48:41.906394: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8904
2023-09-21 15:48:42.952695: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-09-21 15:48:43.178287: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-09-21 15:48:43.183746: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x3a40a80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-21 15:48:43.183793: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA RTX A5000, Compute Capability 8.6
2023-09-21 15:48:43.192094: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-21 15:48:43.257980: I tensorf

73/73 [==============================] - ETA: 0s - loss: 0.4300 - mean_absolute_error: 0.5414
Epoch 1: val_loss improved from inf to 0.19021, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/BRNO_4layers_age%100(linear)_mse(decay)_3fold_test500/batch256,layer1:c128filt11str2,layer2:c256filt13str4,layer3:c32filt5str2,mha:nhead3hsize256,dropout0.2,dnodes16,dropout0.1,lr0.0005/weights_0.hdf5
73/73 [==============================] - 22s 174ms/step - loss: 0.4300 - mean_absolute_error: 0.5414 - val_loss: 0.1902 - val_mean_absolute_error: 0.3801
Epoch 2/100
73/73 [==============================] - ETA: 0s - loss: 0.0806 - mean_absolute_error: 0.2249
Epoch 2: val_loss improved from 0.19021 to 0.09802, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/BRNO_4layers_age%100(linear)_mse(decay)_3fold_test500/batch256,layer1:c128filt11str2,layer2:c256filt13str4,layer3:c32filt5str2,mha:nhead3hsize256,dropout0.2,dnodes16,dropout0.1,lr0.0005/weights_0

2023-09-21 15:52:46.874013: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 470.50MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-09-21 15:52:46.874568: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 198.75MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-09-21 15:52:57.732968: W tensorflow/tsl/framework/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 30.00MiB (rounded to 31457280)requested by op gradient_tape/model/multi_head_attention/einsum_1/Einsum_1
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocat

Graph execution error:

Detected at node 'gradient_tape/model/multi_head_attention/einsum_1/Einsum_1' defined at (most recent call last):
    File "/opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 736, in start
      self.io_loop.start()
    File "/opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio

In [ ]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(Y_test, y_pred)